In [1]:
import os
from operator import itemgetter    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

import tensorflow as tf

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
 
from keras.datasets import imdb

In [2]:
!pip install transformers

In [3]:

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
num_words=10000)

In [4]:
print(train_data)
print(train_labels)

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

In [5]:
word_index = imdb.get_word_index()

reverse_word_index = dict(
[(value, key) for (key, value) in word_index.items()])

decoded_review = ' '.join(
[reverse_word_index.get(i - 3, '?') for i in train_data[123]])

print(decoded_review)

? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too


In [6]:
model_version = 'bert-base-cased'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
ac_dict = {"positive" : 1, "negative" : 0}

In [8]:
import torch.nn as nn

In [9]:

%%capture
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_version,
                                          bos_token = "[CLS]",
                                          eos_token = "[SEP]")
transformer_model = AutoModel.from_pretrained(model_version, output_attentions = True).to(device)
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)
cross_entropy_layer = nn.CrossEntropyLoss()

In [10]:
from tqdm import tqdm

In [11]:
!pip install datasets

In [12]:
from typing import List, Tuple

In [13]:
def pad_batch(elems: List[List[int]], pad_token_id: int) -> List[List[int]]:
    """Pads all lists in elems to the maximum list length of any list in 
    elems. Pads with pad_token_id.
    """
    max_len = max([len(elem) for elem in elems])
    return [elem+[pad_token_id]*(max_len-len(elem)) for elem in elems]

def load_dataset(tokenizer):
  (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
  word_index = imdb.get_word_index()

  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

  new_train_data = []
  for tokenized_text in tqdm(train_data):
    decoded_sent = " ".join([reverse_word_index.get(word - 3, "?") for word in tokenized_text])
    # print(decoded_sent)
    new_encoded_text = tokenizer.encode(decoded_sent)
    new_train_data.append(new_encoded_text)
  
  new_test_data = []
  for tokenized_text in tqdm(test_data):
    decoded_sent = " ".join([reverse_word_index[word] for word in tokenized_text])
    new_encoded_text = tokenizer.encode(decoded_sent)
    new_test_data.append(new_encoded_text)

  return (new_train_data, train_labels), (new_test_data, test_labels)

def generator(dataset_data, dataset_label, max_len = 512, batch_size = 16):
  i = 0
  tokenized_threads, labels = [], []
  while i<len(dataset_data):
    tokenized_threads.append(dataset_data[i][:max_len])
    labels.append(dataset_label[i])
    i += 1
        
    if i%batch_size==0:
      yield (pad_batch(tokenized_threads, tokenizer.pad_token_id), 
                  labels)
      tokenized_threads, labels = [], []

In [14]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters()),
                       lr = 2e-5,)

In [15]:
(train_dataset, train_labels), (test_dataset, test_labels) = load_dataset(tokenizer)

100%|██████████| 25000/25000 [00:20<00:00, 1238.99it/s]


In [16]:
BATCH_SIZE = 16

In [17]:
def train(dataset_data, dataset_label, batch_size):
    global values_weight, values_bias;
    accumulate_over = 4
    
    optimizer.zero_grad()
    print("Training")
    for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label,  batch_size = batch_size)):
        
        #Cast to PyTorch tensor
        tokenized_threads = torch.tensor(tokenized_threads, device=device)
        labels = torch.tensor(labels, device=device, dtype=torch.long)


        loss = compute((tokenized_threads, 
                        labels,), False) / batch_size
        
        print("Loss: ", loss)

        loss.backward()

        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [18]:
from datasets import load_metric
metric1 = load_metric("accuracy")
metric3 = load_metric("precision")
metric4 = load_metric("f1")
metric2 = load_metric("recall")


In [19]:
def evaluate(dataset_data, dataset_label, BATCH_SIZE):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    print('Evaluation')
    
    with torch.no_grad():
        for i, (tokenized_threads, labels) in enumerate(generator(dataset_data, dataset_label, batch_size = BATCH_SIZE)):
            # print(comp_type_labels)
            #Cast to PyTorch tensor
            tokenized_threads = torch.tensor(tokenized_threads, device=device)
            labels = torch.tensor(labels, device=device)

            preds = compute((tokenized_threads, 
                            labels,), pred=True)
            
            
            metric1.add_batch(predictions=preds, 
                            references=labels,)
                            #tokenized_threads=tokenized_threads.cpu().tolist())
            metric2.add_batch(predictions=preds, 
                            references=labels,)
            metric3.add_batch(predictions=preds, 
                            references=labels,)
            metric4.add_batch(predictions=preds, 
                            references=labels,)
            break
        
    print(metric1.compute())
    print(metric2.compute())
    print(metric3.compute())
    print(metric4.compute())

In [20]:

(train_dataset_data, train_dataset_label), (test_dataset_data, test_dataset_label) = load_dataset(tokenizer)

100%|██████████| 25000/25000 [00:20<00:00, 1237.88it/s]


In [21]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor], pred: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, labels = batch
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    print(transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,).last_hidden_state.shape)
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                                            attention_mask=pad_mask,).last_hidden_state)[:, 0, :]
    if(pred):
      return torch.argmax(logits, dim = 1)


    ce_loss = cross_entropy_layer(logits, labels)

    return ce_loss

In [22]:
import numpy as np
def shuffle(data, labels):
  idx = np.random.permutation(len(data))
  x,y = np.array(data)[idx], np.array(labels)[idx]
  return x.tolist(), y.tolist()

In [23]:
n_epochs = 35
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset_data, train_dataset_label = shuffle(train_dataset_data, train_dataset_label)
    test_dataset_data, test_dataset_label = shuffle(test_dataset_data, test_dataset_label)
    train(train_dataset_data, train_dataset_label, BATCH_SIZE)
    evaluate(test_dataset_data, test_dataset_label, BATCH_SIZE)

------------EPOCH 1---------------
Training
torch.Size([16, 512, 768])
Loss:  tensor(0.0428, device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([16, 512, 768])
Loss:  tensor(0.0394, device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([16, 512, 768])
Loss:  tensor(0.0450, device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([16, 366, 768])
Loss:  tensor(0.0439, device='cuda:0', grad_fn=<DivBackward0>)


RuntimeError: ignored